If use Colab set a GPU environment and run cells markdown required by Colab

In [1]:
# Colab required
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Colab required
# change direction to drive/MyDrive/nils/Early_Baseline_PatentMatch_Paragraph_Classification_Mateusz repo folder
%cd drive/MyDrive/nils/Early_Baseline_PatentMatch_Paragraph_Classification_Mateusz

/content/drive/MyDrive/nils/Early_Baseline_PatentMatch_Paragraph_Classification_Mateusz


In [3]:
# Colab required
# update libraries after every runtime reset
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [4]:
import sys
import os
# Add src as folder from where to import
parent_dir = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.', 'src'))

# Add this directory to sys.path
sys.path.append(parent_dir)

In [5]:
from src import LLMmanager, utils
import mlflow
# this code uploads the credentials to MLflow given in the .env file
from src.settings import (
    MLFLOW_TRACKING_URI,
    MLFLOW_TRACKING_USERNAME,
    MLFLOW_TRACKING_PASSWORD,
)

if MLFLOW_TRACKING_URI is None:
    print('importing mlflow credentials failed')

experiment_name = 'Mateusz_PatentMatchBaseline_test'

# If you have provided the name of an experiment that does not exist or that existed in MLflow but was deleted, create a new experiment.
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None or experiment.lifecycle_stage == 'deleted':
    mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment_by_name(experiment_name)

In [6]:
# define model parameters
config = {
        'learning_rate': 2e-5,
        'batch_size': 32,
        'num_epochs': 4,
        'max_length': 512,
        'test_size': 0.1,
        'random_state': 42,
    }

In [7]:
# initialize Trainer
trainer = LLMmanager.TextSimilarityLLMManager(
        model_name='bert-base-uncased',
    tokenizer_name = 'bert-base-uncased',
        config=config,
    verbose=True,
    MLFlow_reporting=True
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU found!


In [8]:
with mlflow.start_run(experiment_id=experiment.experiment_id):
  # set run name
  mlflow.set_tag(key='mlflow.runName',
                       value=f'{utils.timestamp()}_{MLFLOW_TRACKING_USERNAME}')
  trainer.run(test_path='data/test_dataset.json', train_path='data/train_dataset.json')
mlflow.end_run()

Train data len: 450
Validation data len: 50
Validation Accuracy: 0.44
Validation F1 Score: 0.6111111111111112
Validation Matthews Correlation Coefficient: 0.0
Validation Confusion Matrix:
[[ 0 28]
 [ 0 22]]
Validation Accuracy: 0.5
Validation F1 Score: 0.6268656716417911
Validation Matthews Correlation Coefficient: 0.16116459280507606
Validation Confusion Matrix:
[[ 4 24]
 [ 1 21]]
Validation Accuracy: 0.46
Validation F1 Score: 0.6197183098591549
Validation Matthews Correlation Coefficient: 0.12662932291827406
Validation Confusion Matrix:
[[ 1 27]
 [ 0 22]]
Validation Accuracy: 0.56
Validation F1 Score: 0.576923076923077
Validation Matthews Correlation Coefficient: 0.148039131365948
Validation Confusion Matrix:
[[13 15]
 [ 7 15]]
Test Accuracy: 0.5438144329896907
Test F1 Score: 0.556390977443609
Test Matthews Correlation Coefficient: 0.08898744757167883
Test Confusion Matrix:
[[200 194]
 [160 222]]


In [9]:
MLFLOW_TRACKING_URI

'https://mlflow.logosai.tech/'

In [10]:
ls data

test_dataset.json  test.parquet  train_dataset.json  train.parquet
